In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
from collections import Counter

pd.set_option('display.max_columns', None)
EN_train = 'EN/train'
SG_train = 'SG/train'
CN_train = 'CN/train'
EN_test = 'EN/dev.in'
SG_test = 'SG/dev.in'
CN_test = 'CN/dev.in'

In [2]:
def load_train(training_file):
    df = pd.read_csv(training_file, sep=' ', header = None, error_bad_lines=False)
    df.columns=['word','state']
    return df

def load_test(test_file):
    ls = []
    f = open(test_file,encoding="utf8")
    for line in f:
        ls.append(line.strip('\n'))
    df_test = pd.DataFrame(ls)
    df_test.columns=['word']
    return df_test
        
# df_test = load_test(EN_test)
# print(len(df_test))
df_train = load_train(EN_train)
print(len(df_train))
df_train.head(5)

df_test = load_test(EN_test)
print(len(df_test))
df_test.head(5)

181628
27225


,word
0,HBO
1,has
2,close
3,to
4,24


In [3]:
def createMatrix(df):
    start = time.time()
    columns = df.word.unique().tolist()
    index = df.state.unique().tolist()
    new_df = pd.DataFrame(columns=columns, index=index)
    print(f'time elapsed {time.time()-start} seconds')
    return new_df
empty_matrix = createMatrix(df_train)    
# emission_matrix.head(1)

time elapsed 0.8995108604431152 seconds


In [4]:
print(len(empty_matrix))
print(len(empty_matrix.columns))

21
18212


In [5]:
def emissionMatrix_special(df, emission_matrix):
    k=0.5
    start = time.time()
    df_denominator = df.groupby('state').count()   # getting counts of states
    df_counts = df.groupby(['state','word']).size().reset_index()   # getting counts of every word in each state
    df_merged = df_counts.merge(df_denominator, left_on=['state'], right_on='state')  # merge 
    df_merged = df_merged.rename(columns={"word_x": "word",0:"word_count", "word_y": "state_count"})
    df_merged['Probability'] = df_merged.word_count/(df_merged.state_count+k)    # get emission probability (count of word in that state/ state count)
    for index, row in tqdm(df_merged.iterrows()):  # for every known probabilty
        emission_matrix.loc[row['state'],row['word']] = row['Probability']   # append into the emission matrix
    for i in df_train.state.unique().tolist():
        emission_matrix.loc[i,'#UNK#'] = float(k/df_denominator.loc[i]+k)
    emission_matrix = emission_matrix.fillna(0)   # fill those null cells with zero
    print(f'time elapsed {time.time()-start}')
    return emission_matrix

emission_matrix = emissionMatrix_special(df_train, empty_matrix)

25051it [04:08, 100.79it/s]
time elapsed 250.04122018814087


In [6]:
def argmax(df):
    start = time.time()
    tags={}
    for col in df.columns:
        tags[col]=[df[col].argmax()]
    return tags
        
tags = argmax(emission_matrix)

def tag_system(tag_dict, test_df):
    start = time.time()
    test_ls = test_df['word'].tolist()
    tag_states=[]
    for i in test_ls:
        if i in tag_dict.keys():
            tag_states.append(tag_dict[i])
        elif i=="":   # for blank lines, set state to be blank
            tag_states.append("")
        elif i not in tag_dict.keys():
            tag_states.append(tag_dict['#UNK#'])

    test_df['states']=tag_states
    print(f'time elapsed {time.time()-start}')
    return test_df
output = tag_system(tags,df_test)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
output.head(50)

In [ ]:
def load_train_trans(training_file):
    f = open(training_file)
    ls_state = ['START']
    for line in f:
        item = line.strip('\n').split(' ')
        if len(item) == 2:
            ls_state.append(item[1])
        elif len(item) < 2:
            ls_state.append('STOP')
            ls_state.append('START')
    ls_state.pop(-1)
    return ls_state

def relation_matrix(temp):
    count = Counter(temp)
    list_key = list(count.keys())
    rls_matrix = pd.DataFrame(columns=list_key, index=list_key)
    for (x, y), c in Counter(zip(temp, temp[1:])).items():
        rls_matrix.loc[[x], [y]] = c/count[x]
    rls_matrix = rls_matrix.fillna(value=0)
    rls_matrix = rls_matrix.drop(columns='START')
    rls_matrix = rls_matrix.drop(index='STOP')
    return rls_matrix

In [ ]:
sequence_ls = load_train_trans(EN_train)
transition_matrix = relation_matrix(sequence_ls)

In [ ]:
print(transition_matrix.head())

In [ ]:
# emission_matrix; transition_matrix

In [ ]:
m = open('EN/dev.in', encoding="utf8")
ls=[]
big_ls=[]
for line in m:
    item=line.strip('\n')
    if item=='':
        big_ls.append(ls)
        ls=[]
    elif item!='':
        ls.append(item)

In [ ]:
Multiple=[] # will conrain
for i in big_ls:  #for each sentene
    Vertibri=[] # will contain weights for all layers within each document
    forward_steps = len(i)+1 
    for j in forward_steps: #for each layer
        if j==0:  #start to first layer
            layer_v= [a*b for a,b in zip(transition_matrix.,[all states --> i[j] #word in sentence])]
            big_v.append(small_v)
        elif j!=0 & j!=forward_steps: #not first or last step
            small_v=[a*b*c for a,b,c in zip([small_v],[state-> state],[ state -> word])]
            big_v.append(small_v)
        else: #if last step

In [ ]:
def log(x, inf_replace=-1000):
    out = np.log(x)
    out[~np.isfinite(out)] = inf_replace
    return out
logged_emission = log(emission_matrix)
logged_transition = log(transition_matrix)
transition_np = logged_transition.drop(['START']).drop('STOP',axis=1).to_numpy()

# test for one document
tags = argmax(emission_matrix)   # vocab of words
Vertibri = []
document = big_ls[1]
# print(document)
forward_steps = len(document)+1
for i in range(forward_steps):
    if i == 0: # for from START to first layer
        if document[i] in tags.keys():
            layer = [t+e for t,e in zip(logged_transition.loc['START'].drop('STOP'), logged_emission[document[i]])]
        elif document[i] not in tags.keys():
            layer = [t+e for t,e in zip(logged_transition.loc['START'].drop('STOP'), logged_emission['#UNK#'])]
        Vertibri.append(layer)
        print(type(Vertibri[-1]))
    elif i!=0 and i!=forward_steps-1: #not first or last step
        prev_layer_prob = Vertibri[-1]*21
        prev_layer_prob = np.array(prev_layer_prob).reshape(21,21).T
        m = prev_layer_prob + transition_np
        if document[i] in tags.keys():
            emission_ls = logged_emission[document[i]].tolist()*21
            emission_np = np.array(emission_ls).reshape(21,21)
        elif document[i] not in tags.keys():
            emission_ls = logged_emission['#UNK#'].tolist()*21
            emission_np = np.array(emission_ls).reshape(21,21)
        matrix = (m + emission_np)
        layer = np.amax(matrix,0)
        Vertibri.append(layer.tolist())
    elif i == forward_steps-1:
        prev_layer_prob = np.array(Vertibri[-1])
        last = logged_transition.drop('START')['STOP'].tolist()
        layer = prev_layer_prob+last
        Vertibri.append(layer.tolist())

state_order = []
states = emission_matrix.index.tolist()
# Vertibri.pop(0)
for layer in Vertibri:
    position = layer.index(max(layer))
    state_order.append(states[position])
# state_order = []
# states = emission_matrix.index.tolist()
# for layer in Vertibri:
#     position = layer.index[max(layer)]
#     # states = emission_matrix.index.tolist()
#     state_order.append(states[position])
# print(state_order)

In [ ]:
print(Vertibri.pop(0))
for layer in Vertibri:
    print(layer.index(max(layer)))

In [ ]:
state_order = []
states = emission_matrix.index.tolist()
Vertibri.pop(0)
for layer in Vertibri:
    position = layer.index(max(layer))
    # states = emission_matrix.index.tolist()
    state_order.append(states[position])
print(state_order)

In [ ]:
print(len(Vertibri))
print(Vertibri[11])
print(Vertibri[11].index(max(Vertibri[11])))

In [ ]:
prev_layer_prob = np.array(Vertibri[-2])
print(prev_layer_prob)
print(logged_transition.drop('START')['STOP'].tolist())

print(prev_layer_prob + logged_transition.drop('START')['STOP'].tolist() )

In [ ]:
# transition_np = logged_transition.drop(['START']).drop('STOP',axis=1).to_numpy()
# ls = Vertibri[-1]*21
# prev_np = np.array(ls).reshape(21,21).T
# print(prev_np)
# print(Vertibri[-1].tolist()*21)
# ls = logged_emission[document[3]].tolist()*21
# emission_np = np.array(ls).reshape(21,21)
# print(emission_np)

print(logged_transition.drop('START').index.tolist())
print(emission_matrix.index.tolist())

In [ ]:
# tags = argmax(emission_matrix)   # vocab of words
# Vertibri = []
# document = big_ls[1]
# # print(document)
# forward_steps = len(document)+1
# for i in range(forward_steps):
#     if i == 0: # for from START to first layer
#         if document[i] in tags.keys():
#             layer = [t*e for t,e in zip(transition_matrix.loc['START'].drop('STOP'), emission_matrix[document[i]])]
#         elif document[i] not in tags.keys():
#             layer = [t*e for t,e in zip(transition_matrix.loc['START'].drop('STOP'), emission_matrix['#UNK#'])]
#         Vertibri.append(layer)
#         break
# print(Vertibri[0])
# print(Vertibri[0].index(max(Vertibri[0])))

In [ ]:
document = big_ls[1]
len(document)

In [ ]:
import numpy as np
ls = [1/6]*3 + [0]*3 + [1/16]*3
# print(ls)
prev = np.array(ls).reshape(3,3)

ls1 = [1/6,0,4/6,1/4,0,0,1/8,4/8,1/8]
trans = np.array(ls1).reshape(3,3)

ls2 = [1/6,1/4,1/8]*3
print(len(ls))
em = np.array(ls2).reshape(3,3)
print('prev prob matrix')
print(prev)
print('trans matrix')
print(trans)
print('em matrix')
print(em)

output = np.multiply(prev,trans)
output = np.multiply(output,em)
print('next prob matrix')
print(output)

print(np.amax(output,0))


In [ ]:
df = pd.DataFrame(
	[[21, 72, 67],
	[23, 78, 69],
	[32, 74, 56],
	[52, 54, 76]],
	columns=['a', 'b', 'c'])

print('DataFrame\n----------\n', df)

#convert dataframe to numpy array
arr = df.to_numpy()

print('\nNumpy Array\n----------\n', arr)

print(arr.shape)